In [1]:
import requests
from io import BytesIO
import rasterio
from rasterio.warp import transform_bounds
import xml.etree.ElementTree as ET
from pystac.extensions.table import TableExtension
from shapely.geometry import mapping, box


In [24]:


state = "uttar_pradesh"
district = "jaunpur"
block = "badlapur"

GEOSERVER_WCS_URL = "https://geoserver.core-stack.org:8443/geoserver/LULC_level_1/wcs"
coverage_id = "LULC_level_1:LULC_23_24_badlapur_level_1"

params = {
    "service": "WCS",
    "version": "2.0.1",
    "request": "GetCoverage",
    "CoverageId": coverage_id,
    #"bbox": bbox,
    "format": "geotiff"
}


In [25]:
response = requests.get(GEOSERVER_WCS_URL, params=params, verify=False)
response.raise_for_status()
raster_data = BytesIO(response.content)

with rasterio.open(raster_data) as src:
    bbox = src.bounds
    print(f"Bounding Box: {bbox}")

/home/vishnu/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'geoserver.core-stack.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


KeyboardInterrupt: 

In [26]:
params = {
    "service": "WCS",
    "version": "2.0.1",
    "request": "GetCoverage",
    "CoverageId": coverage_id,
    "format": "geotiff"
}

try:

    response = requests.get(GEOSERVER_WCS_URL, params=params, verify=False)
    response.raise_for_status()

    raster_data = BytesIO(response.content)
            


    with rasterio.open(raster_data) as src:
        bounds = src.bounds
        geom = mapping(box(*bounds))
        data_type = str(src.dtypes[0])
        height, width = src.shape
        data_crs = src.crs

        proj_epsg = None
        if src.crs and src.crs.is_epsg_code:
            proj_epsg = src.crs.to_epsg()

        print(f"EPSG: {proj_epsg} ")
        print(f"Image Dimensions:")
        print(f"Width: {width} pixels")
        print(f"Height: {height} pixels") 
        
        if proj_epsg != 7755:
            reprojected_bounds = transform_bounds(src.crs, 'EPSG:7755', *bounds)
            bbox = list(reprojected_bounds)
            gsd_x = (reprojected_bounds[2] - reprojected_bounds[0]) / width
            gsd_y = (reprojected_bounds[3] - reprojected_bounds[1]) / height
            gsd = (gsd_x + gsd_y) / 2
        else:
            bbox = [bounds.left, bounds.bottom, bounds.right, bounds.top]
            gsd = src.res[0]
            
    print(f"Raster resolution (GSD): {gsd} meters")  

except requests.exceptions.RequestException as e:
    print(f"An error occurred while fetching data from GeoServer: {e}")     

/home/vishnu/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'geoserver.core-stack.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


EPSG: 4326 
Image Dimensions:
Width: 5442 pixels
Height: 3150 pixels
Raster resolution (GSD): 9.474690935940632 meters


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, Normalize
import requests
import os
import xml.etree.ElementTree as ET
from io import BytesIO

def parse_qml_classes_from_url(qml_url):
    try:
        response = requests.get(qml_url, verify=False)
        response.raise_for_status()
        
    
        tree = ET.parse(BytesIO(response.content))
        root = tree.getroot()
        classes = []

        for entry in root.findall(".//paletteEntry"):
            class_info = {}
            for attr_key, attr_value in entry.attrib.items():
                if attr_key == "value":
                    try:
                        class_info[attr_key] = int(attr_value)
                    except ValueError:
                        class_info[attr_key] = attr_value
                else:
                    class_info[attr_key] = attr_value
            classes.append(class_info)

        if not classes:
            for entry in root.findall(".//item"):
                class_info = {}
                for attr_key, attr_value in entry.attrib.items():
                    if attr_key == "value":
                        try:
                            class_info[attr_key] = int(attr_value)
                        except ValueError:
                            class_info[attr_key] = attr_value
                    else:
                        class_info[attr_key] = attr_value
                classes.append(class_info)
        return classes

    except requests.exceptions.RequestException as e:
        print(f"Error fetching QML file from URL: {e}")
        return None
    except ET.ParseError as e:
        print(f"Error parsing XML from QML file: {e}")
        return None

github_raw_qml_url = "https://raw.githubusercontent.com/core-stack-org/QGIS-Styles/main/Land/LULC0_12class.qml"

style_info = parse_qml_classes_from_url(github_raw_qml_url)

if style_info:
    print("Parsed QML classes from GitHub:")
    for cls in style_info:
        print(cls)

/home/vishnu/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'raw.githubusercontent.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Parsed QML classes from GitHub:
{'value': 0, 'label': 'clear', 'alpha': '0', 'color': '#000000'}
{'value': 1, 'label': 'built up', 'alpha': '255', 'color': '#ff0000'}
{'value': 2, 'label': 'kharif water', 'alpha': '255', 'color': '#74ccf4'}
{'value': 3, 'label': 'kharif and rabi water', 'alpha': '255', 'color': '#1ca3ec'}
{'value': 4, 'label': 'kharif and rabi and zaid water', 'alpha': '255', 'color': '#0f5e9c'}
{'value': 5, 'label': 'crops', 'alpha': '255', 'color': '#f1c232'}
{'value': 6, 'label': 'trees', 'alpha': '255', 'color': '#38761d'}
{'value': 7, 'label': 'barren land', 'alpha': '255', 'color': '#a9a9a9'}
{'value': 8, 'label': 'single kharif cropping', 'alpha': '255', 'color': '#bad93e'}
{'value': 9, 'label': 'single non-kharif Cropping', 'alpha': '255', 'color': '#f59d22'}
{'value': 10, 'label': 'double cropping', 'alpha': '255', 'color': '#ff9371'}
{'value': 11, 'label': 'tripple/annual/perennial cropping', 'alpha': '255', 'color': '#b3561d'}
{'value': 12, 'label': 'shrubs 

In [ ]:
def generate_raster_thumbnail(raster_data_stream, out_path, style_info):
    with rasterio.open(raster_data_stream) as src:
        arr = src.read(1)
        nodata = src.nodata
    
    if nodata is not None:
        arr = np.ma.masked_equal(arr, nodata)

    values = [cls['value'] for cls in style_info if 'value' in cls]
    colors = [cls['color'] for cls in style_info if 'color' in cls]
    
    value_to_color = {int(v): c for v, c in zip(values, colors)}
    sorted_values = sorted(value_to_color.keys())
    
    cmap = ListedColormap([value_to_color.get(v, '#808080') for v in sorted_values])
    norm = plt.Normalize(vmin=min(sorted_values), vmax=max(sorted_values))
    
    fig, ax = plt.subplots(1, 1, figsize=(3, 3), dpi=100)
    ax.imshow(arr, cmap=cmap, norm=norm)
    plt.axis('off')
    
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    plt.savefig(out_path, bbox_inches='tight', pad_inches=0)
    plt.close()


GEOSERVER_WCS_URL = "https://geoserver.core-stack.org:8443/geoserver/LULC_level_1/wcs"
coverage_id = "LULC_level_1:LULC_23_24_badlapur_level_1"


qml_url = "https://raw.githubusercontent.com/core-stack-org/QGIS-Styles/main/Land/LULC0_12class.qml"
thumbnail_output_path = "/home/vishnu/thumbnails/lulc_raster_thumbnail.png"


params = {
    "service": "WCS",
    "version": "2.0.1",
    "request": "GetCoverage",
    "CoverageId": coverage_id,
    "format": "geotiff"
}

response = requests.get(GEOSERVER_WCS_URL, params=params, verify=False)
response.raise_for_status()
raster_data = BytesIO(response.content)


style_info = parse_qml_classes_from_url(qml_url)


if style_info:
    generate_raster_thumbnail(raster_data, thumbnail_output_path, style_info)
    print(f"Thumbnail generated and saved to: {thumbnail_output_path}")
else:
    print("Could not generate thumbnail due to error parsing QML style information.")


/home/vishnu/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'geoserver.core-stack.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/vishnu/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'raw.githubusercontent.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Thumbnail generated and saved to: /home/vishnu/thumbnails/lulc_raster_thumbnail.png


### To fetch the data from geoserver for vector layer (admin boundary) and make in tabular format.

In [5]:
import geopandas as gpd
import os
import requests
import pystac
from datetime import datetime, timezone
import pandas as pd
from shapely.geometry import mapping, box
from pystac.extensions.table import TableExtension
vector_desc_df = pd.DataFrame(columns=['column_name', 'column_description'])

In [6]:

geojson_url = "https://geoserver.core-stack.org:8443/geoserver/panchayat_boundaries/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=panchayat_boundaries%3Ajaunpur_badlapur&maxFeatures=50&outputFormat=application%2Fjson"


In [7]:
response = requests.get(geojson_url)
response.raise_for_status()

In [8]:
gdf = gpd.read_file(response.text)
print(gdf.head())


                   id  ADI_2001  ADI_2011  ADI_2019  ASSET_2001  ASSET_2011  \
0  jaunpur_badlapur.1         0         0         0           0           0   
1  jaunpur_badlapur.2         0         0         0           0           0   
2  jaunpur_badlapur.3         0         0         0           0           0   
3  jaunpur_badlapur.4         0         0         0           0           0   
4  jaunpur_badlapur.5         0         0         0           0           0   

   ASSET_2019 BF_2001  BF_2011  BF_2019  ...  TOT_P  block_cen  dist_cen  \
0           0       0        0        0  ...      0          0         0   
1           0       0        0        0  ...      0          0         0   
2           0       0        0        0  ...      0          0         0   
3           0       0        0        0  ...      0          0         0   
4           0       0        0        0  ...      0          0         0   

   district          state  state_cen    tehsil  vill_ID    vill_nam

In [9]:
gdf_wgs84 = gdf.to_crs(epsg=4326) if gdf.crs is None or gdf.crs.to_epsg() != 4326 else gdf

bounds = gdf_wgs84.total_bounds
bbox = [float(b) for b in bounds]
geom = mapping(gdf_wgs84.union_all())

In [10]:
item_id = os.path.splitext(os.path.basename(geojson_url))[0]
item = pystac.Item(
        id=item_id,
        bbox=list(bbox),
        geometry=geom,
        datetime=datetime.now(timezone.utc),
        properties={
            "title": "title"
        }
    )

In [11]:
vector_merged_df = gdf.dtypes.reset_index()
vector_merged_df.columns = ['column_name', 'column_dtype']
vector_merged_df = vector_merged_df.merge(vector_desc_df, on='column_name', how='left').fillna('')




In [12]:
column_data = [
    {
        "name": row['column_name'],
        "type": str(row['column_dtype']),
        "description": row['column_description']
    }
    for _, row in vector_merged_df.iterrows()
]

In [13]:
table_ext = TableExtension.ext(item, add_if_missing=True)
table_ext.columns = column_data


In [14]:
print("Tabular data of the STAC Item's columns:")
tabular_output = pd.DataFrame(column_data)
print(tabular_output)

Tabular data of the STAC Item's columns:
          name      type description
0           id    object            
1     ADI_2001     int32            
2     ADI_2011     int32            
3     ADI_2019     int32            
4   ASSET_2001     int32            
5   ASSET_2011     int32            
6   ASSET_2019     int32            
7      BF_2001    object            
8      BF_2011     int32            
9      BF_2019     int32            
10     FC_2001     int32            
11     FC_2011     int32            
12     FC_2019     int32            
13       F_ILL     int32            
14       F_LIT     int32            
15        F_SC     int32            
16        F_ST     int32            
17    MSW_2001     int32            
18    MSW_2011     int32            
19    MSW_2019     int32            
20       M_ILL     int32            
21       M_LIT     int32            
22        M_SC     int32            
23        M_ST     int32            
24       No_HH     int32          